In [ ]:
import tensorflow as tf


In [ ]:
print(tf.__version__)

In [ ]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

from sklearn.model_selection import train_test_split

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings



warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data=pd.read_csv('favorites_sample_small.csv', delimiter=';',error_bad_lines=False)


In [ ]:
data.head(10)

In [ ]:
data.info()

In [ ]:
data['Description_Flag'] = np.where(data['pet_description'].notnull(), 1, 0)



In [ ]:
data['organization_type'].fillna("Unknown", inplace = True) 

In [ ]:
data.head(10)

In [ ]:
print("Unique Values in `Flag_Detail` => {}".format(data.Description_Flag.unique()))

In [ ]:
data_des = data[[
    'Description_Flag'
    ,'pet_description']].copy()

data_des.head()

In [ ]:
data2 = data[[
    'user_id'
    ,'id'
    ,'num_aifs']].copy()

In [ ]:
datatest = data[[
    'user_id'
    ,'id'
    ,'Description_Flag'
    ,'mixed_breed'
    ,'animal_type'
    ,'age'
    ,'state_code'
    ,'primary_breed'
    ,'organization_type'
    ,'gender'
    ,'size'
    ,'num_aifs'
    ,'house_trained'
    ,'coat_length'
    ,'num_videos'
    ,'num_photos'
    ,'good_with_dogs'
    ,'good_with_cats'
    ,'good_with_children']].copy()

In [ ]:
n_users = len(data2.user_id.unique())
print(n_users)
n_favorites = len(data2.id.unique())
print(n_favorites)

In [ ]:
train, test = train_test_split(data2, test_size=0.2, random_state=42)


In [ ]:
print (train)


In [ ]:
# convert "user_id" and "id" column to sequential id's
train['user_id'] = train.groupby(train.user_id).grouper.group_info[0]
train['id'] = train.groupby(train.id).grouper.group_info[0]

In [ ]:

fav_input = Input(shape=[1], name="Fav-Input")
fav_embedding = Embedding(n_favorites, 50, name="Fav-Embedding")(fav_input)
fav_vec = Flatten(name="Flatten-Books")(fav_embedding)


user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users, 50, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)


prod = Dot(name="Dot-Product", axes=1)([fav_vec, user_vec])
model = Model([user_input, fav_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
#Model error
#https://github.com/tensorflow/tensorflow/issues/23698


from keras.models import load_model

if os.path.exists('model.h5'):
    model = load_model('model.h5')
else:
    history = model.fit([train.user_id, train.id], train.num_aifs, epochs=10, verbose=1)
    model.save('model.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")